training_set = Tot setul de date, este vectorul care contine toate documentele
document = Element din training_set. Fiecare document contine "data", "annotations" si "predictions"


In [332]:
import json

# Step 1: Specify the file path
file_path = "./train_data.json"  # Replace with your actual file path

# Step 2: Open the file in read mode
try:
  with open(file_path, "r") as json_file:
    # Step 3: Load the JSON data using json.load()
    training_set = json.load(json_file)
except FileNotFoundError:
  print(f"Error: File not found at {file_path}")
else:
  print("JSON data parsed successfully!")
  # Step 4: Access and process the data
  # (See examples below based on data structure)

print(training_set[0])


JSON data parsed successfully!
{'data': {'cmbd': 'null', 'id': '19026587', 'docid': 'null', 'page': 'null', 'paragraph': 'null', 'text': " nº historia clinica: ** *** *** nºepisodi: ******** sexe: home data de naixement: 16.05.1936 edat: 82 anys procedencia cex mateix hosp servei urologia data d'ingres 24.07.2018 data d'alta 25.07.2018 08:54:04 ates per ***************, *****; ****************, ****** informe d'alta d'hospitalitzacio motiu d'ingres paciente que ingresa de forma programada para realizacion de uretrotomia interna . antecedents alergia a penicilina y cloramfenicol . no habitos toxicos. antecedentes medicos: bloqueo auriculoventricular de primer grado hipertension arterial. diverticulosis extensa insuficiencia renal cronica colelitiasis antecedentes quirurgicos: exeresis de lesiones cutaneas con anestesia local protesis total de cadera cordectomia herniorrafia inguinal proces actual varon de 81a que a raiz de episodio de hematuria macroscopica se realiza cistoscopia que es

Construim doua seturi care sa contina toate cuvintele care exprima negatia si incertitudinea si alte doua seturi care sa retina scopurile.

In [333]:
NEG = set()
UNC = set()
NSCO = set()
USCO = set()


In [334]:
# Gets a list of tuples representing character offsets and returns list of words
def get_words(text, offsets):
  words = []
  for start, end in offsets:
    words.append(text[start:end])
  return words

In [335]:
# Parses a document and returns 4 lists of words
def find_cues_and_scopes(document):
  neg_postitions_pairs = [(result_element["value"]["start"], result_element["value"]["end"]) for result_element in document["predictions"][0]["result"] if "NEG" in result_element["value"]["labels"]]
  unc_postitions_pairs = [(result_element["value"]["start"], result_element["value"]["end"]) for result_element in document["predictions"][0]["result"] if "UNC" in result_element["value"]["labels"]]
  nsco_postitions_pairs = [(result_element["value"]["start"], result_element["value"]["end"]) for result_element in document["predictions"][0]["result"] if "NSCO" in result_element["value"]["labels"]]
  usco_postitions_pairs = [(result_element["value"]["start"], result_element["value"]["end"]) for result_element in document["predictions"][0]["result"] if "USCO" in result_element["value"]["labels"]]
  neg_words = get_words(document["data"]["text"], neg_postitions_pairs)
  unc_words = get_words(document["data"]["text"], unc_postitions_pairs)
  nsco_words = get_words(document["data"]["text"], nsco_postitions_pairs)
  usco_words = get_words(document["data"]["text"], usco_postitions_pairs) 
  return neg_words, unc_words, nsco_words, usco_words

Cream seturile de cuvinte

In [336]:
num_nsco = 0
num_usco = 0
for i,document in enumerate(training_set):
  neg_words, unc_words, nsco_words, usco_words = find_cues_and_scopes(document)
  nsco_words_set = set(nsco_words)
  usco_words_set = set(usco_words)
  num_nsco += len(nsco_words)
  num_usco += len(usco_words)

  NEG.update(neg_words)
  UNC.update(unc_words)
  NSCO.update(nsco_words)
  USCO.update(usco_words)

# Removing spaces and punctation signs from the start and end of each string
NEG = {word.strip(" ,.!?;)") for word in NEG}
UNC = {word.strip(" ,.!?);") for word in UNC}
NSCO = {word.strip(" ,.!?;)") for word in NSCO}
USCO = {word.strip(" ,.!?);") for word in USCO}

print("Neg words: ", len(NEG))
print("Example: ", list(NEG)[:10])
print("Unc words: ", len(UNC))
print("Example: ", list(UNC)[:10])
print("Nsco words: ", len(NSCO))
print("Example: ", list(NSCO)[:10])
print("Usco words: ", len(USCO))
print("Example: ", list(USCO)[:10])


Neg words:  58
Example:  ['retirar', 'se retira', 'desorientacion', 'excepto', 'negatividad', 'impide', 'negativa', 'neg', 'asintomatico', 'cede']
Unc words:  79
Example:  ['podrian', 'valorar', 'clara', 'sospechosa de', 'sospechan de', 'orientan', 'no parece', 'podria', 'sugestiva como', 'impresiona']
Nsco words:  2250
Example:  ['levantar pesos ni realizar ejercicio fisico', 'percepcion de mal descanso', 'refiere sangrado', 'cansancio', 'signos de meningitis', 'tto', 'hay litiasis en el trayecto ureteral', 'hematomas', 'signos de vasculitis', 'evidenciar a area de nefritis ni imagenes que sugieran colecciones']
Usco words:  391
Example:  ['neoplasia', 'secundaria a hipoxemia cronica', 'apendicitis aguda', 'tambien izquierda', 'isquemia', 'conservada', 'el cuadro como una sepsis de origen urinario', 'neoformacion visualizada por endoscopia', 'infiltracion de pancreas', 'contactar con el canal del nervio dentario']


Incepem sa testam metoda baseline pe test_data.

In [337]:
import json

# Step 1: Specify the file path
file_path = "./test_data.json"  # Replace with your actual file path

# Step 2: Open the file in read mode
try:
  with open(file_path, "r") as json_file:
    # Step 3: Load the JSON data using json.load()
    test_set = json.load(json_file)
except FileNotFoundError:
  print(f"Error: File not found at {file_path}")
else:
  print("JSON data parsed successfully!")
  # Step 4: Access and process the data
  # (See examples below based on data structure)

print(test_set[0])


JSON data parsed successfully!
{'data': {'cmbd': 'null', 'id': '20306671', 'docid': 'null', 'page': 'null', 'paragraph': 'null', 'text': " nº historia clinica: ** *** *** nºepisodi: ******** sexe: dona data de naixement: 12.05.1977 edat: 42 anys procedencia aguts servei obstetricia data d'ingres 27.09.2019 data d'alta 01.10.2019 13:00:00 ates per *****************, **************; *****************, ***** informe d'alta d'hospitalitzacio motiu d'ingres induccion al parto por pequeño para la edad gestacional ( peg ) antecedents no alergias medicamentosas conocidas antcededentes medico-quirurgicos: protesis mamaria, adenoidectomia niega habitos toxicos medicacio habitual anafranil25 mg/ diario. yodocefol. hierro oral. ranitidina 150 mg. proces actual edad: 42 años tpal: 1001 (eutocico) fur: 21.12.2019 sg: 40+0 gsrh:a+ gestacion controlada en alto riesgo obstetrico (aro) por antecedente de depresion postparto + diabetes gestacional ( dga1 ) + pequeño para la edad gestacional (peg) secundi

DARIAN REGEX

In [338]:
# text = orice text din test_set
# set_of_words = NEG, UNC, NSCO, USCO

import re

def make_prediction(text, set_of_words):
    pattern = "|".join(set_of_words)
    regex = rf"\b({pattern})\b"
    matches = re.finditer(regex, text)

    if matches:
        for match in matches:
            # Get the matched word and its starting/ending positions
            matched_word = match.group(1)
            start_pos = match.start()
            end_pos = match.end()
            if text[end_pos] == " ":
                end_pos += 1
            print(f"Found '{matched_word}' at positions ({start_pos}, {end_pos})")

    else:
        print("No neg words from the list found in the text.")

In [339]:
# Predict for the first document in the test set
make_prediction(test_set[0]["data"]["text"], NEG)

Found 'no' at positions (395, 398)
Found 'niega' at positions (499, 505)
Found 'negativo' at positions (1111, 1119)
Found 'no' at positions (1141, 1144)
Found 'no' at positions (1163, 1166)
Found 'negativos' at positions (1194, 1203)
Found 'negativo' at positions (1313, 1322)
Found 'sin' at positions (2118, 2122)


In [340]:
import re

text = test_set[0]["data"]["text"]

# Join words into a pipe-separated pattern
NEG_pattern = "|".join(NEG)
UNC_pattern = "|".join(UNC)
NSCO_pattern = "|".join(NSCO)
USCO_pattern = "|".join(USCO)


# Construct the regex with word boundaries
neg_regex = rf"\b({NEG_pattern})\b"
unc_regex = rf"\b({UNC_pattern})\b"
nsco_regex = rf"\b({NSCO_pattern})\b"
usco_regex = rf"\b({USCO_pattern})\b"

# Search for matches
neg_matches = re.finditer(neg_regex, text)
unc_matches = re.finditer(unc_regex, text)
#nsco_matches = re.findall(nsco_regex, text)
#usco_matches = re.findall(usco_regex, text)

if neg_matches:
  for match in neg_matches:
    # Get the matched word and its starting/ending positions
    matched_word = match.group(1)
    start_pos = match.start()
    end_pos = match.end()
    print(f"Found '{matched_word}' at positions ({start_pos}, {end_pos})")
  #print("Found words:", neg_matches)
else:
  print("No neg words from the list found in the text.")

if unc_matches:
  for match in unc_matches:
    # Get the matched word and its starting/ending positions
    matched_word = match.group(1)
    start_pos = match.start()
    end_pos = match.end()
    print(f"Found '{matched_word}' at positions ({start_pos}, {end_pos})")
  #print("Found words:", unc_matches)
else:
  print("No unc words from the list found in the text.")




Found 'no' at positions (395, 397)
Found 'niega' at positions (499, 504)
Found 'negativo' at positions (1111, 1119)
Found 'no' at positions (1141, 1143)
Found 'no' at positions (1163, 1165)
Found 'negativos' at positions (1194, 1203)
Found 'negativo' at positions (1313, 1321)
Found 'sin' at positions (2118, 2121)
Found 'no' at positions (395, 397)
Found 'no' at positions (1141, 1143)
Found 'no' at positions (1163, 1165)
Found 'sin' at positions (2118, 2121)
Found 'puede' at positions (3460, 3465)


In [341]:
# Parseaza obicetele din vectorul "result" din vector si returneaza 4 liste de tupluri cu informatii despre fiecare element
# Orice tuplu din lista e de forma (tip_de_label, start, end, id). 
# tip_de_label apartine {NEG, UNC, NSCO, USCO}

def get_results_from_document(document):
    neg_predictions, unc_predictions, nsco_predictions, usco_predictions = [], [], [], []
    for result_element in document["predictions"][0]["result"]:
        if "NEG" in result_element["value"]["labels"]:
            neg_predictions.append(("NEG",result_element["value"]["start"], result_element["value"]["end"], result_element["id"]))
        if "UNC" in result_element["value"]["labels"]:
            unc_predictions.append(("UNC",result_element["value"]["start"], result_element["value"]["end"], result_element["id"]))
        if "NSCO" in result_element["value"]["labels"]:
            nsco_predictions.append(("NSCO",result_element["value"]["start"], result_element["value"]["end"], result_element["id"]))
        if "USCO" in result_element["value"]["labels"]:
            usco_predictions.append(("USCO",result_element["value"]["start"], result_element["value"]["end"], result_element["id"]))

    return neg_predictions, unc_predictions, nsco_predictions, usco_predictions

In [342]:
# Get ground truth from document
# If category is empty, it will return all categories
def print_ground_truth(document, category=[]): 
    neg_words, unc_words, nsco_words, usco_words = find_cues_and_scopes(document)
    neg_results, unc_results, nsco_results, usco_results = get_results_from_document(document)
    if not category:
        print("NEG")
        for word, result in zip(neg_words, neg_results):
            print(f"Found '{word}' at positions ({result[1]}, {result[2]})")
        print("UNC")
        for word, result in zip(unc_words, unc_results):
            print(f"Found '{word}' at positions ({result[1]}, {result[2]})")
        print("NSCO")
        for word, result in zip(nsco_words, nsco_results):
            print(f"Found '{word}' at positions ({result[1]}, {result[2]})")
        print("USCO")
        for word, result in zip(usco_words, usco_results):
            print(f"Found '{word}' at positions ({result[1]}, {result[2]})")
    else:
        if "NEG" in category:
            print("NEG")
            for word, result in zip(neg_words, neg_results):
                print(f"Found '{word}' at positions ({result[1]}, {result[2]})")
        if "UNC" in category:
            print("UNC")
            for word, result in zip(unc_words, unc_results):
                print(f"Found '{word}' at positions ({result[1]}, {result[2]})")
        if "NSCO" in category:
            print("NSCO")
            for word, result in zip(nsco_words, nsco_results):
                print(f"Found '{word}' at positions ({result[1]}, {result[2]})")
        if "USCO" in category:
            print("USCO")
            for word, result in zip(usco_words, usco_results):
                print(f"Found '{word}' at positions ({result[1]}, {result[2]})")
    

In [343]:

for i, document in enumerate(test_set):
  print("Document ", i, "Ground Truth:")
  print_ground_truth(document, ["NEG"])
  print("Document ", i, "Predictions:")
  make_prediction(document["data"]["text"], NEG)

Document  0 Ground Truth:
NEG
Found 'no ' at positions (395, 398)
Found 'niega ' at positions (499, 505)
Found 'no ' at positions (1141, 1144)
Found 'no ' at positions (1163, 1166)
Found 'sin ' at positions (2118, 2122)
Found 'negativo' at positions (1111, 1119)
Found 'negativos' at positions (1194, 1203)
Document  0 Predictions:
Found 'no' at positions (395, 398)
Found 'niega' at positions (499, 505)
Found 'negativo' at positions (1111, 1119)
Found 'no' at positions (1141, 1144)
Found 'no' at positions (1163, 1166)
Found 'negativos' at positions (1194, 1203)
Found 'negativo' at positions (1313, 1322)
Found 'sin' at positions (2118, 2122)
Document  1 Ground Truth:
NEG
Found 'no ' at positions (377, 380)
Found 'niega ' at positions (446, 452)
Found 'sin ' at positions (1307, 1311)
Found 'no ' at positions (1348, 1351)
Found 'sin ' at positions (1784, 1788)
Found 'sin ' at positions (1886, 1890)
Found 'sin ' at positions (1960, 1964)
Found 'niega ' at positions (2001, 2007)
Found 'sin ' 

Below I am trying to gather all words found in scopes into a single regex pattern, and use it to find scopes.

In [344]:
# Combine negation and uncertainty scopes into a single set
ALL_SCOPES = NSCO.union(USCO)
ALL_SCOPES = {word.strip(" ,.!?*;)") for word in ALL_SCOPES}
ALL_SCOPES = {word for word in ALL_SCOPES if "*" not in word and "(" not in word and ")" not in word}

print(len(ALL_SCOPES))      
print(len(NSCO))            
print(len(USCO))

# A set with all individual words from the scopes
SCOPE_words = set()         # ['erc', '(29/05/18)', 'ser', 'visibles', 'extratono', 'inicia', 'valor', 'frialdad', 'medicamentoses', 'neoformativo']
for scope in ALL_SCOPES:
  SCOPE_words.update(scope.split())
  
print("NEG_UNC words before processing: ", len(SCOPE_words))
  
# Remove all symbols and numbers from the set
SCOPE_words = {word for word in SCOPE_words if word.isalpha()}
  
print("NegScopes: ", len(NSCO))
print("UScopes: ", len(USCO))
print("NEG_UNC words: ", len(SCOPE_words))
print("Example: ", list(SCOPE_words)[:30])

2588
2250
391
NEG_UNC words before processing:  3148
NegScopes:  2250
UScopes:  391
NEG_UNC words:  2871
Example:  ['secundarios', 'hemorragia', 'neoplasia', 'cansancio', 'tto', 'traquea', 'metenamina', 'nasal', 'cardiologia', 'articular', 'hematomas', 'es', 'impulsividad', 'sinusal', 'acompañante', 'resecable', 'isquemia', 'cetosis', 'conservada', 'tumor', 'hiperseñal', 'dificutlad', 'complicaciones', 'quirurgicos', 'va', 'hipotiroïdisme', 'picos', 'inicialmente', 'exudado', 'e']


In [345]:
UNC = {word for word in UNC if word not in NEG}

In [346]:
import re

predictions = []

for document in test_set:
    neg_list, unc_list, usco_list, nsco_list = [], [], [], []
    text = document["data"]["text"]
    # text = 'no signos lesiones'

    # Join words into a pipe-separated pattern
    NEG_pattern = "|".join(NEG)
    UNC_pattern = "|".join(UNC)

    # Join scopes into a pipe-separated pattern
    SCOPE_pattern = "|".join(ALL_SCOPES)

    # Split SCOPE_pattern into individual words
    words = SCOPE_pattern.split('|')

    # Sort the words in descending order of length, so the regex find the longest match
    words.sort(key=len, reverse=True)

    # Join the sorted words into a regex pattern
    SCOPE_pattern = '|'.join(words)

    # Construct the regex with word boundaries
    neg_regex = rf"\b({NEG_pattern})\b"
    unc_regex = rf"\b({UNC_pattern})\b"

    neg_w_regex = rf"\b({NEG_pattern})\b\s+(\w+)"
    unc_w_regex = rf"\b({UNC_pattern})\b\s+(\w+)"

    # Regex that finds the negation, and the next 5 words from SCOPE_pattern
    neg_sc_regex = rf"\b({NEG_pattern})\b\s+((?:\b(?:{SCOPE_pattern})\b\s*){{0,5}})"
    unc_sc_regex = rf"\b({UNC_pattern})\b\s+((?:\b(?:{SCOPE_pattern})\b\s*){{0,5}})"

    # Search for matches
    neg_matches = re.finditer(neg_w_regex, text)
    unc_matches = re.finditer(unc_w_regex, text)

    # Search for matches (neg + word + scope words)
    neg_scopes_matches = re.finditer(neg_sc_regex, text)
    unc_scopes_matches = re.finditer(unc_sc_regex, text)
    
   
    for match in neg_scopes_matches:
        # Get the matched word and its starting/ending positions
        matched_word = match.group(1)
        start_pos = match.start(1)
        end_pos = match.end(1)+1
        #print(f"Found '{matched_word}' at positions ({start_pos}, {end_pos})")
        neg_list.append((start_pos, end_pos, matched_word))
        # # Get the scope word
        scope_word = match.group(2)
        sc_start_pos = match.start(2)
        sc_end_pos = match.end(2)+1
        nsco_list.append((sc_start_pos, sc_end_pos, scope_word))
        # print(f"Found scope '{scope_word}' at positions ({sc_start_pos}, {sc_end_pos})")
        # print("Whole match:", match.group(0))

    for match in unc_scopes_matches:
        # Get the matched word and its starting/ending positions
        matched_word = match.group(1)
        start_pos = match.start(1)
        end_pos = match.end(1)+1
        unc_list.append((start_pos, end_pos, matched_word))
       # print(f"Found '{matched_word}' at positions ({start_pos}, {end_pos})")
        # # Get the scope word
        scope_word = match.group(2)
        sc_start_pos = match.start(2)
        sc_end_pos = match.end(2)+1
        usco_list.append((sc_start_pos, sc_end_pos, scope_word))
        # print(f"Found scope '{scope_word}' at positions ({sc_start_pos}, {sc_end_pos})")
        # print("Whole match:", match.group(0))

    neg_list = sorted(neg_list, key=lambda x: x[0])
    unc_list = sorted(unc_list, key=lambda x: x[0])
    nsco_list = sorted(nsco_list, key=lambda x: x[0])
    usco_list = sorted(usco_list, key=lambda x: x[0])
    predictions.append({"NEG": neg_list, "UNC": unc_list, "NSCO": nsco_list, "USCO": usco_list})

print(predictions[0])
print("no" in UNC)


{'NEG': [(395, 398, 'no'), (499, 505, 'niega'), (1141, 1144, 'no'), (1163, 1166, 'no'), (1313, 1322, 'negativo'), (2118, 2122, 'sin')], 'UNC': [(3460, 3466, 'puede')], 'NSCO': [(398, 433, 'alergias medicamentosas conocidas '), (505, 541, 'habitos toxicos medicacio habitual '), (1144, 1151, 'inmune'), (1166, 1173, 'immune'), (1322, 1323, ''), (2122, 2134, 'incidencias')], 'USCO': [(3466, 3467, '')]}
False


Analizam NEG, UNC, NSCO si USCO pentru test_set si facem intersectia lor cu train_set ca sa vedem cam ce procentaj de cuvinte se repeta.

In [349]:
NEG2 = set()
UNC2 = set()
NSCO2 = set()
USCO2 = set()

In [350]:
num_nsco2 = 0
num_usco2 = 0
neg2_dict = {}
unc2_dict = {}
nsco2_dict = {}
usco2_dict = {}
total_negs = 0
total_nsco = 0
total_uncs = 0
total_uscos = 0

for i,document in enumerate(test_set):
  neg_words, unc_words, nsco_words, usco_words = find_cues_and_scopes(document)
  
  # Verific numarul total de apartitii (neunice)
  total_negs += len(neg_words)
  total_nsco += len(nsco_words)
  total_uncs += len(unc_words)
  total_uscos += len(usco_words)
  for word in neg_words:
    word = word.strip(" ,.!?;)")
    if word in neg2_dict:
      neg2_dict[word] += 1
    else:
      neg2_dict[word] = 1
  for word in unc_words:
    word = word.strip(" ,.!?;)")
    if word in unc2_dict:
      unc2_dict[word] += 1
    else:
      unc2_dict[word] = 1
  for word in nsco_words:
    word = word.strip(" ,.!?;)")
    if word in nsco2_dict:
      nsco2_dict[word] += 1
    else:
      nsco2_dict[word] = 1
  for word in usco_words:
    word = word.strip(" ,.!?;)")
    if word in usco2_dict:
      usco2_dict[word] += 1
    else:
      usco2_dict[word] = 1

  nsco_words_set = set(nsco_words)
  usco_words_set = set(usco_words)
  num_nsco2 += len(nsco_words)
  num_usco2 += len(usco_words)

  NEG2.update(neg_words)
  UNC2.update(unc_words)
  NSCO2.update(nsco_words)
  USCO2.update(usco_words)

# Removing spaces and punctation signs from the start and end of each string
NEG2 = {word.strip(" ,.!?;)") for word in NEG2}
UNC2 = {word.strip(" ,.!?);") for word in UNC2}
NSCO2 = {word.strip(" ,.!?;)") for word in NSCO2}
USCO2 = {word.strip(" ,.!?);") for word in USCO2}

print("Neg words: ", len(NEG2))
print("Example: ", list(NEG2)[:10])
print("Unc words: ", len(UNC2))
print("Example: ", list(UNC2)[:10])
print("Nsco words: ", len(NSCO2))
print("Example: ", list(NSCO2)[:10])
print("Usco words: ", len(USCO2))
print("Example: ", list(USCO2)[:10])


1074
791
123
129
Neg words:  38
Example:  ['retirar', 'retirado', 'negativa', 'neg', 'asintomatico', 'cede', 'inespecifica', 'sense', 'exfumador', 'niega']
Unc words:  44
Example:  ['', 'sospecha de', 'sin claro', 'valorar', 'sugestivas de', 'sospechosa de', 'puede', 'poco sugestivo de', 'posibles', 'dudosamente']
Nsco words:  698
Example:  ['haber requerido nuevos ingresos', 'explica alteraciones del ritmo deposicional ni sindrome miccional', 'dolor toracico', 'signos de malignidad', 'hemocultivos 17/09', 'se observan otras lesiones focales sospechosas', 'varices', 'alteraciones agudas', 'disartria', 'hay reserva']
Usco words:  121
Example:  ['cronico', 'corresponder con saco gestacional desestructurado', 'manejo activo', 'que su enolismo cronico e hipovitaminosis hayan tenido un papel en la clinica de la paciente', 'prerenal', 'engrosamiento mural de recto proximal / sigma', 'saos moderado medicacio habitual', 'lesion pulmonar', 'en 1/2013', 'gestacion ectopica']


In [351]:
print("Common NEG words: ", len(NEG.intersection(NEG2)), " out of ", len(NEG), " and ", len(NEG2))
print("Common UNC words: ", len(UNC.intersection(UNC2)), " out of ", len(UNC), " and ", len(UNC2))
print("Common NSCO words: ", len(NSCO.intersection(NSCO2)), " out of ", len(NSCO), " and ", len(NSCO2))
print("Common USCO words: ", len(USCO.intersection(USCO2)), " out of ", len(USCO), " and ", len(USCO2))

Common NEG words:  27  out of  58  and  38
Common UNC words:  32  out of  77  and  44
Common NSCO words:  243  out of  2250  and  698
Common USCO words:  10  out of  391  and  121


In [352]:
valid = 0
for word,num_app in neg2_dict.items():
  # Atentie. NEG este setul cuvintelor din TRAINING SET. Eu aici verific cate cuvinte din test set sunt in training set
  if word in NEG:
    valid += num_app

print("Valid neg words: ", valid, " out of ", total_negs)

Valid neg words:  1118  out of  1132


In [353]:
valid = 0
for word,num_app in unc2_dict.items():
  # Atentie. NEG este setul cuvintelor din TRAINING SET. Eu aici verific cate cuvinte din test set sunt in training set
  if word in UNC:
    valid += num_app

print("Valid unc words: ", valid, " out of ", total_uncs)

Valid unc words:  118  out of  131


In [354]:
valid = 0
for word,num_app in nsco2_dict.items():
  # Atentie. NEG este setul cuvintelor din TRAINING SET. Eu aici verific cate cuvinte din test set sunt in training set
  if word in NSCO:
    valid += num_app

print("Valid neg scopes: ", valid, " out of ", total_nsco)

Valid neg scopes:  599  out of  1074


In [355]:
valid = 0
for word,num_app in nsco2_dict.items():
  # Atentie. NEG este setul cuvintelor din TRAINING SET. Eu aici verific cate cuvinte din test set sunt in training set
  if word in ALL_SCOPES:
    valid += num_app

print("Valid neg scopes: ", valid, " out of ", total_nsco)

Valid neg scopes:  599  out of  1074


In [356]:
valid = 0
for word,num_app in usco2_dict.items():
  # Atentie. NEG este setul cuvintelor din TRAINING SET. Eu aici verific cate cuvinte din test set sunt in training set
  if word in USCO:
    valid += num_app

print("Valid unc scopes: ", valid, " out of ", total_uscos)

Valid unc scopes:  11  out of  129


In [357]:
valid = 0
for word,num_app in usco2_dict.items():
  # Atentie. NEG este setul cuvintelor din TRAINING SET. Eu aici verific cate cuvinte din test set sunt in training set
  if word in ALL_SCOPES:
    valid += num_app

print("Valid unc scopes: ", valid, " out of ", total_uscos)

Valid unc scopes:  20  out of  129


In [358]:
# Gets a list of words and 
def get_gt_format(document):
    neg_predictions, unc_predictions, nsco_predictions, usco_predictions = [], [], [], []
    text = document["data"]["text"]
    for result_element in document["predictions"][0]["result"]:
        start = result_element["value"]["start"]
        end = result_element["value"]["end"]
        if "NEG" in result_element["value"]["labels"]:   
            neg_predictions.append((start, end, text[start:end]))
        if "UNC" in result_element["value"]["labels"]:
            unc_predictions.append((start, end, text[start:end]))
        if "NSCO" in result_element["value"]["labels"]:
            nsco_predictions.append((start, end, text[start:end]))
        if "USCO" in result_element["value"]["labels"]:
            usco_predictions.append((start, end, text[start:end]))

    return neg_predictions, unc_predictions, nsco_predictions, usco_predictions

In [359]:
# FORMAT : (NEG, START, END, WORD)
def get_ground_truth(document):
    neg_results, unc_results, nsco_results, usco_results = get_gt_format(document)

    neg_results_sorted = sorted(neg_results, key=lambda x: x[0])
    unc_results_sorted = sorted(unc_results, key=lambda x: x[0])
    nsco_results_sorted = sorted(nsco_results, key=lambda x: x[0])
    usco_results_sorted = sorted(usco_results, key=lambda x: x[0])

    ground_truth_dict = {"NEG": neg_results_sorted, "UNC": unc_results_sorted, "NSCO": nsco_results_sorted, "USCO": usco_results_sorted}

    return ground_truth_dict



get_ground_truth(test_set[0])


{'NEG': [(395, 398, 'no '),
  (499, 505, 'niega '),
  (1111, 1119, 'negativo'),
  (1141, 1144, 'no '),
  (1163, 1166, 'no '),
  (1194, 1203, 'negativos'),
  (2118, 2122, 'sin ')],
 'UNC': [],
 'NSCO': [(398, 422, 'alergias medicamentosas '),
  (505, 521, 'habitos toxicos '),
  (1107, 1111, 'vih '),
  (1144, 1150, 'inmune'),
  (1166, 1172, 'immune'),
  (1174, 1194, 'lues vih, vhb y vhc '),
  (2122, 2133, 'incidencias')],
 'USCO': []}

In [361]:
# List of dictionaries of GT docuemnts in the test set
ground_truths = [get_ground_truth(document) for document in test_set]
print(ground_truths[2])
  

{'NEG': [(621, 627, 'niega '), (766, 770, 'sin '), (1028, 1032, 'sin '), (1548, 1554, 'niega '), (1596, 1602, 'niega '), (1618, 1624, 'niega '), (1662, 1666, 'sin '), (2027, 2031, 'sin '), (2134, 2138, 'sin '), (2146, 2150, 'sin '), (2266, 2269, 'no '), (2464, 2468, 'sin '), (2534, 2538, 'sin '), (2567, 2571, 'sin '), (3229, 3233, 'sin '), (3321, 3325, 'sin '), (3333, 3337, 'sin '), (3453, 3456, 'no '), (3693, 3696, 'no '), (3823, 3827, 'sin '), (3883, 3887, 'sin '), (3896, 3899, 'no '), (3954, 3958, 'sin '), (4262, 4265, 'no '), (4850, 4858, 'negativo'), (4860, 4863, 'no '), (4882, 4890, 'negativo'), (4983, 4986, 'no '), (5350, 5354, 'sin '), (5421, 5425, 'sin '), (5609, 5613, 'sin '), (5978, 5981, 'no '), (6034, 6037, 'no '), (6056, 6059, 'no '), (7507, 7517, 'negativos.'), (7664, 7672, 'negativa'), (7752, 7756, 'sin '), (7882, 7885, 'no '), (8083, 8086, 'no '), (8275, 8282, 'afebril'), (8388, 8392, 'sin '), (8542, 8546, 'sin '), (8641, 8645, 'sin '), (8958, 8965, 'afebril'), (9535, 

DE TRANSFORMAT IN FUNCTIE BLOCURILE ASTEA DOUA DE JOS CARE RULEAZA PE UN ANUMIT REGEX 

TEST 1 REGEX PETRU

In [367]:
import re

predictions = []

for document in test_set:
    neg_list, unc_list, usco_list, nsco_list = [], [], [], []
    text = document["data"]["text"]
    # text = 'no signos lesiones'

    # Join words into a pipe-separated pattern
    NEG_pattern = "|".join(NEG)
    UNC_pattern = "|".join(UNC)

    # Join scopes into a pipe-separated pattern
    SCOPE_pattern = "|".join(ALL_SCOPES)

    # Split SCOPE_pattern into individual words
    words = SCOPE_pattern.split('|')

    # Sort the words in descending order of length, so the regex find the longest match
    words.sort(key=len, reverse=True)

    # Join the sorted words into a regex pattern
    SCOPE_pattern = '|'.join(words)

    # Construct the regex with word boundaries
    neg_regex = rf"\b({NEG_pattern})\b"
    unc_regex = rf"\b({UNC_pattern})\b"

    neg_w_regex = rf"\b({NEG_pattern})\b\s+(\w+)"
    unc_w_regex = rf"\b({UNC_pattern})\b\s+(\w+)"

    # Regex that finds the negation, and the next 5 words from SCOPE_pattern
    neg_sc_regex = rf"\b({NEG_pattern})\b\s+((?:\b(?:{SCOPE_pattern})\b\s*){{0,5}})"
    unc_sc_regex = rf"\b({UNC_pattern})\b\s+((?:\b(?:{SCOPE_pattern})\b\s*){{0,5}})"

    # Search for matches
    neg_matches = re.finditer(neg_w_regex, text)
    unc_matches = re.finditer(unc_w_regex, text)

    # Search for matches (neg + word + scope words)
    neg_scopes_matches = re.finditer(neg_sc_regex, text)
    unc_scopes_matches = re.finditer(unc_sc_regex, text)
    
   
    for match in neg_scopes_matches:
        # Get the matched word and its starting/ending positions
        matched_word = match.group(1)
        start_pos = match.start(1)
        end_pos = match.end(1)+1
        #print(f"Found '{matched_word}' at positions ({start_pos}, {end_pos})")
        neg_list.append((start_pos, end_pos, matched_word))
        # # Get the scope word
        scope_word = match.group(2)
        sc_start_pos = match.start(2)
        sc_end_pos = match.end(2)+1
        nsco_list.append((sc_start_pos, sc_end_pos, scope_word))
        # print(f"Found scope '{scope_word}' at positions ({sc_start_pos}, {sc_end_pos})")
        # print("Whole match:", match.group(0))

    for match in unc_scopes_matches:
        # Get the matched word and its starting/ending positions
        matched_word = match.group(1)
        start_pos = match.start(1)
        end_pos = match.end(1)+1
        unc_list.append((start_pos, end_pos, matched_word))
       # print(f"Found '{matched_word}' at positions ({start_pos}, {end_pos})")
        # # Get the scope word
        scope_word = match.group(2)
        sc_start_pos = match.start(2)
        sc_end_pos = match.end(2)+1
        usco_list.append((sc_start_pos, sc_end_pos, scope_word))
        # print(f"Found scope '{scope_word}' at positions ({sc_start_pos}, {sc_end_pos})")
        # print("Whole match:", match.group(0))

    neg_list = sorted(neg_list, key=lambda x: x[0])
    unc_list = sorted(unc_list, key=lambda x: x[0])
    nsco_list = sorted(nsco_list, key=lambda x: x[0])
    usco_list = sorted(usco_list, key=lambda x: x[0])
    predictions.append({"NEG": neg_list, "UNC": unc_list, "NSCO": nsco_list, "USCO": usco_list})

print(predictions[0])
print("no" in UNC)


{'NEG': [(395, 398, 'no'), (499, 505, 'niega'), (1141, 1144, 'no'), (1163, 1166, 'no'), (1313, 1322, 'negativo'), (2118, 2122, 'sin')], 'UNC': [(3460, 3466, 'puede')], 'NSCO': [(398, 433, 'alergias medicamentosas conocidas '), (505, 541, 'habitos toxicos medicacio habitual '), (1144, 1151, 'inmune'), (1166, 1173, 'immune'), (1322, 1323, ''), (2122, 2134, 'incidencias')], 'USCO': [(3466, 3467, '')]}
False


In [368]:
for d1,d2 in zip(predictions[1:2],ground_truths[1:2]):
  precision = {"NEG":0,"NSCO":0,"UNC":0,"USCO":0}
  recall = {"NEG":0,"NSCO":0,"UNC":0,"USCO":0}
  f1 = {"NEG":0,"NSCO":0,"UNC":0,"USCO":0}
  print(d1["UNC"])
  print(d2["UNC"])
  for key in d1:
    #print(key)
    tp = {"NEG":0,"NSCO":0,"UNC":0,"USCO":0}
    for elem in d1[key]:
      for elem2 in d2[key]:
        if abs(elem[0]-elem2[0]) <= 1 and abs(elem[1]-elem2[1]) <=1:
          tp[key]+=1
          break
  
    precision[key] = tp[key]/len(d1[key])
    #recall[key] = tp[key]/len(d2[key])
    #f1[key] = 2*precision[key]*recall[key]/(precision[key]+recall[key])


print(precision)
print(recall)
print(f1)

[(5653, 5663, 'descartar'), (7080, 7087, 'podria'), (7091, 7102, 'compatible'), (7396, 7404, 'posible')]
[(7080, 7087, 'podria '), (7396, 7404, 'posible ')]
{'NEG': 0.9642857142857143, 'NSCO': 0.4642857142857143, 'UNC': 0.5, 'USCO': 0.25}
{'NEG': 0, 'NSCO': 0, 'UNC': 0, 'USCO': 0}
{'NEG': 0, 'NSCO': 0, 'UNC': 0, 'USCO': 0}


TEST 2 REGEX DARIAN

In [369]:
import re

predictions = []

for document in test_set:
    neg_list, unc_list, usco_list, nsco_list = [], [], [], []
    text = document["data"]["text"]
    # text = 'no signos lesiones'

    # Join words into a pipe-separated pattern
    NEG_pattern = "|".join(NEG)
    UNC_pattern = "|".join(UNC)

    # Join scopes into a pipe-separated pattern
    SCOPE_pattern = "|".join(ALL_SCOPES)

    # Split SCOPE_pattern into individual words
    words = SCOPE_pattern.split('|')

    # Sort the words in descending order of length, so the regex find the longest match
    words.sort(key=len, reverse=True)

    # Join the sorted words into a regex pattern
    SCOPE_pattern = '|'.join(words)

    # Construct the regex with word boundaries
    neg_regex = rf"\b({NEG_pattern})(\b|[^\w\s]+)"
    unc_regex = rf"\b({UNC_pattern})(\b|[^\w\s]+)"

    neg_w_regex = rf"\b({NEG_pattern})\b\s+(\w+)"
    unc_w_regex = rf"\b({UNC_pattern})\b\s+(\w+)"

    # Regex that finds the negation, and the next 5 words from SCOPE_pattern
    neg_sc_regex = rf"\b({NEG_pattern})\b\s*((?:\b(?:{SCOPE_pattern})\b\s*){{0,5}})"
    unc_sc_regex = rf"\b({UNC_pattern})\b\s*((?:\b(?:{SCOPE_pattern})\b\s*){{0,5}})"

    # Search for matches
    neg_matches = re.finditer(neg_w_regex, text)
    unc_matches = re.finditer(unc_w_regex, text)

    # Search for matches (neg + word + scope words)
    neg_scopes_matches = re.finditer(neg_sc_regex, text)
    unc_scopes_matches = re.finditer(unc_sc_regex, text)
    
   
    for match in neg_scopes_matches:
        # Get the matched word and its starting/ending positions
        matched_word = match.group(1)
        start_pos = match.start(1)
        end_pos = match.end(1)+1
        #print(f"Found '{matched_word}' at positions ({start_pos}, {end_pos})")
        neg_list.append((start_pos, end_pos, matched_word))
        # # Get the scope word
        scope_word = match.group(2)
        sc_start_pos = match.start(2)
        sc_end_pos = match.end(2)+1
        nsco_list.append((sc_start_pos, sc_end_pos, scope_word))
        # print(f"Found scope '{scope_word}' at positions ({sc_start_pos}, {sc_end_pos})")
        # print("Whole match:", match.group(0))

    for match in unc_scopes_matches:
        # Get the matched word and its starting/ending positions
        matched_word = match.group(1)
        start_pos = match.start(1)
        end_pos = match.end(1)+1
        unc_list.append((start_pos, end_pos, matched_word))
       # print(f"Found '{matched_word}' at positions ({start_pos}, {end_pos})")
        # # Get the scope word
        scope_word = match.group(2)
        sc_start_pos = match.start(2)
        sc_end_pos = match.end(2)+1
        usco_list.append((sc_start_pos, sc_end_pos, scope_word))
        # print(f"Found scope '{scope_word}' at positions ({sc_start_pos}, {sc_end_pos})")
        # print("Whole match:", match.group(0))

    neg_list = sorted(neg_list, key=lambda x: x[0])
    unc_list = sorted(unc_list, key=lambda x: x[0])
    nsco_list = sorted(nsco_list, key=lambda x: x[0])
    usco_list = sorted(usco_list, key=lambda x: x[0])
    predictions.append({"NEG": neg_list, "UNC": unc_list, "NSCO": nsco_list, "USCO": usco_list})

print(predictions[0])
print("no" in UNC)


{'NEG': [(395, 398, 'no'), (499, 505, 'niega'), (1111, 1120, 'negativo'), (1141, 1144, 'no'), (1163, 1166, 'no'), (1194, 1204, 'negativos'), (1313, 1322, 'negativo'), (2118, 2122, 'sin')], 'UNC': [(3460, 3466, 'puede')], 'NSCO': [(398, 433, 'alergias medicamentosas conocidas '), (505, 541, 'habitos toxicos medicacio habitual '), (1119, 1120, ''), (1144, 1151, 'inmune'), (1166, 1173, 'immune'), (1203, 1204, ''), (1322, 1323, ''), (2122, 2134, 'incidencias')], 'USCO': [(3466, 3467, '')]}
False


In [370]:
for d1,d2 in zip(predictions[1:2],ground_truths[1:2]):
  precision = {"NEG":0,"NSCO":0,"UNC":0,"USCO":0}
  recall = {"NEG":0,"NSCO":0,"UNC":0,"USCO":0}
  f1 = {"NEG":0,"NSCO":0,"UNC":0,"USCO":0}
  print(d1["UNC"])
  print(d2["UNC"])
  for key in d1:
    #print(key)
    tp = {"NEG":0,"NSCO":0,"UNC":0,"USCO":0}
    for elem in d1[key]:
      for elem2 in d2[key]:
        if abs(elem[0]-elem2[0]) <= 1 and abs(elem[1]-elem2[1]) <=1:
          tp[key]+=1
          break
  
    precision[key] = tp[key]/len(d1[key])
    #recall[key] = tp[key]/len(d2[key])
    #f1[key] = 2*precision[key]*recall[key]/(precision[key]+recall[key])


print(precision)
print(recall)
print(f1)

[(4029, 4039, 'aparentes'), (5653, 5663, 'descartar'), (7080, 7087, 'podria'), (7091, 7102, 'compatible'), (7383, 7395, 'desconocido'), (7396, 7404, 'posible')]
[(7080, 7087, 'podria '), (7396, 7404, 'posible ')]
{'NEG': 0.967741935483871, 'NSCO': 0.41935483870967744, 'UNC': 0.3333333333333333, 'USCO': 0.16666666666666666}
{'NEG': 0, 'NSCO': 0, 'UNC': 0, 'USCO': 0}
{'NEG': 0, 'NSCO': 0, 'UNC': 0, 'USCO': 0}
